In [119]:
import pandas as pd
import numpy as np

1. 윤아 평균근속연수 -> update
2. ~~K2 score -> update~~

3. 2018 부터 DF to Jiwoo  -> 중분류컬럼 지우기 drop 

In [120]:
df = pd.read_excel('../dataset_final/main_df.xlsx', index_col=0)

In [121]:
df_work = pd.read_excel('../dataset_final/비재무최종_df.xlsx', index_col=0)

In [122]:
df['평균근속연수(년)'] = df_work['평균근속연수']

In [123]:
df_k2 = pd.read_excel('../dataset_final/K2_score_real_final.xlsx', index_col=0)
df_k2

,회사명,거래소코드,회계년도,log총자산(백만원),총자산회전율(log),누적수익성,재무구조,K2_score,K2_부실판단
0,(주)CMG제약,58820,2011/12,10.214832,-0.430596,-1.655914,4.072802,-32.582151,부실가능
1,(주)CMG제약,58820,2012/12,10.650332,-0.954153,-1.156158,5.900594,-21.375391,부실가능
2,(주)CMG제약,58820,2013/12,10.942178,-1.018783,-0.925333,3.727510,-19.041475,부실가능
3,(주)CMG제약,58820,2014/12,10.950639,-0.918205,-0.917659,8.975031,-12.591318,부실가능
4,(주)CMG제약,58820,2015/12,10.956066,-0.923632,-0.949445,19.555639,-1.100565,판정보류
...,...,...,...,...,...,...,...,...,...
17482,흥구석유(주),24060,2018/12,11.258792,0.762205,0.434820,4.900155,14.473588,건전판정
17483,흥구석유(주),24060,2019/12,11.282676,0.640163,0.470459,13.670314,24.934030,건전판정
17484,흥구석유(주),24060,2020/12,11.334892,0.316436,0.501236,19.906224,31.890912,건전판정
17485,흥구석유(주),24060,2021/12,11.328813,0.463362,0.515682,27.143226,40.858411,건전판정


In [124]:
df['K2_score'] = df_k2['K2_score']
df['K2_부실판단'] = df_k2['K2_부실판단']

---

# I. 공통   

In [125]:
df.head(1)

,회사명,거래소코드,회계년도,주거래은행,평균근속연수(년),R&D집중도(%),업력(년),최대주주특수관계인_지분율(%),최대주주변경횟수(회),감사법인변경횟수(회),...,PPI(2015기준),실질GDP성장률(%),외화포지션(%),K2_score,K2_부실판단,EV/EBITDA(배),52주베타(배),PER,PBR,PCR
0,(주)CMG제약,58820,2011,KEB하나은행,3.666667,3.32,10,2.31,0,0,...,105.71,3.7,154.163051,-32.582151,부실가능,18.37,0.070097,0.0,2.21,0.0


## I-1. Target'삭제'기준 Rows -> Drop
    - rows : 17,487 -> 11,248 

In [126]:
main_df = df.loc[~(df['Target']=='삭제')]
main_df.shape

(11248, 62)

## I-2. 컬럼별 info() 확인

In [127]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11248 entries, 0 to 17486
Data columns (total 62 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   회사명                  11248 non-null  object 
 1   거래소코드                11248 non-null  int64  
 2   회계년도                 11248 non-null  int64  
 3   주거래은행                11243 non-null  object 
 4   평균근속연수(년)            11081 non-null  float64
 5   R&D집중도(%)            11141 non-null  float64
 6   업력(년)                11248 non-null  int64  
 7   최대주주특수관계인_지분율(%)     11219 non-null  float64
 8   최대주주변경횟수(회)          11248 non-null  int64  
 9   감사법인변경횟수(회)          11248 non-null  int64  
 10  직원수증감율(%)            11105 non-null  float64
 11  수출기업여부(dummy)        11248 non-null  int64  
 12  기간제비율(%)             11168 non-null  float64
 13  표준산업분류_중분류           7162 non-null   object 
 14  산업위험_경기민감도           11248 non-null  object 
 15  Target               11248 non-null 

In [128]:
# 거래소코드 type 변경 : int -> str
main_df['거래소코드'] = main_df['거래소코드'].astype('str').str.zfill(6)
# main_df['거래소코드']

C:\Users\zino0\AppData\Local\Temp\ipykernel_23268\947282485.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_df['거래소코드'] = main_df['거래소코드'].astype('str').str.zfill(6)


## I-3. col별 처리 

## I-3. columns 조정
- 컬럼 순서 조정
    - 재무 -> 비재무 -> Target 순

In [129]:
main_df_copy = main_df.copy()

In [130]:
main_df = main_df.reindex(columns=[
       # 기본
       '회사명', '거래소코드', '회계년도',
       
       # 안정성
       '유동비율(%)', '당좌비율(%)', '비유동비율(%)',
       '부채비율(%)', '유동부채비율(%)', '차입금의존도(%)', 
       '이익잉여금비율(%)', '순운전자본비율(%)',
       '비유동장기적합률(%)', '이자보상배율(이자비용)(배)', '외화포지션(%)',

       # 수익성
       '총자본순이익률(%)', '자기자본순이익률(%)', '경영자본순이익률(%)', '매출액순이익률(%)',
       '총자본정상영업이익률(%)', '자기자본정상영업이익률(%)', '매출액정상영업이익률(%)', 
       '금융비용부담률(%)',  

       # 활동성
       '총자산회전율(배)','매출채권회전률(배)', '당좌자산회전률(배)',
       '재고자산회전률(배)', '유형자산회전율(배)', '매입채무회전률(배)', 
       '자기자본회전률(배)', '경영자본회전률(배)',
         
       # 성장성
       '총자산_증가율(%)','유동자산증가율(%)',
       '매출액증가율(%)', '정상영업이익증가율(%)', '순이익증가율(%)', 
             
       # 거시
       'spread(%)', 'PPI(2015기준)','실질GDP성장률(%)',

       # 시장정보_주가관련
       'EV/EBITDA(배)','52주베타(배)', 'PER', 'PBR', 'PCR',

       # 기타_재무
       'CASH FLOW 대 부채비율(%)', '총자본투자효율(%)',
       '평균총자산(천원)', '매출액(수익)(천원)',
       'K2_score', 'K2_부실판단',
       
       
       # 비재무정보_기업위험요인
       # 산업위험
       '산업위험_경기민감도','표준산업분류_중분류', 
       
       # 경영위험
       '최대주주특수관계인_지분율(%)', '최대주주변경횟수(회)', '감사법인변경횟수(회)', '업력(년)',

       # 영업위험
       '평균근속연수(년)', '직원수증감율(%)', '기간제비율(%)', 
       'R&D집중도(%)', '수출기업여부(dummy)',
        
       # 기타_비재무
       '주거래은행',

       # Target 
       'Target'])

In [131]:
# 총자산_증가율(%) 제거 - 결측치(2011') 많음 & 성장성 카테고리의 기타 변수 충분하다 판단 
main_df.drop('총자산_증가율(%)', axis=1, inplace=True)
# main_df.head(1)

### 조정 columns

In [132]:
# 기업규모 : 평균총자산 컬럼 log 처리 
main_df['평균총자산(천원)'] = np.log(main_df['평균총자산(천원)'])

In [133]:
# 외화포지션이름 변경 :외화포지션(%) -> 외화포지션(배) 

In [134]:
main_df.columns = [
       # 기본
       '회사명', '거래소코드', '회계년도',
       
       # 안정성
       '유동비율(%)', '당좌비율(%)', '비유동비율(%)',
       '부채비율(%)', '유동부채비율(%)', '차입금의존도(%)', 
       '이익잉여금비율(%)', '순운전자본비율(%)',
       '비유동장기적합률(%)', '이자보상배율(이자비용)(배)', '외화포지션(배)',

       # 수익성
       '총자본순이익률(%)', '자기자본순이익률(%)', '경영자본순이익률(%)', '매출액순이익률(%)',
       '총자본정상영업이익률(%)', '자기자본정상영업이익률(%)', '매출액정상영업이익률(%)', 
       '금융비용부담률(%)',  

       # 활동성
       '총자산회전율(배)','매출채권회전률(배)', '당좌자산회전률(배)',
       '재고자산회전률(배)', '유형자산회전율(배)', '매입채무회전률(배)', 
       '자기자본회전률(배)', '경영자본회전률(배)',
         
       # 성장성
       '유동자산증가율(%)', '매출액증가율(%)', '정상영업이익증가율(%)', '순이익증가율(%)', 
             
       # 거시
       'spread(%)', 'PPI(2015기준)','실질GDP성장률(%)',

       # 시장정보_주가관련
       'EV/EBITDA(배)','52주베타(배)', 'PER', 'PBR', 'PCR',

       # 기타_재무
       'CASH FLOW 대 부채비율(%)', '총자본투자효율(%)',
       'log_평균총자산', '매출액(수익)(천원)',
       'K2_score', 'K2_부실판단',
       
       
       # 비재무정보_기업위험요인
       # 산업위험
       '산업위험_경기민감도','표준산업분류_중분류', 
       
       # 경영위험
       '최대주주특수관계인_지분율(%)', '최대주주변경횟수(회)', '감사법인변경횟수(회)', '업력(년)',

       # 영업위험
       '평균근속연수(년)', '직원수증감율(%)', '기간제비율(%)', 
       'R&D집중도(%)', '수출기업여부(dummy)',
        
       # 기타_비재무
       '주거래은행',

       # Target 
       'Target']

In [135]:
main_df_copy_2 = main_df.copy()

# II. 결측치

In [136]:
# 컬럼별 결측치 Check
for i in main_df.columns : 
    print(f'{i} : {main_df[i].isna().sum()}')
    print('________________________')
    
main_df.shape

회사명 : 0
________________________
거래소코드 : 0
________________________
회계년도 : 0
________________________
유동비율(%) : 96
________________________
당좌비율(%) : 96
________________________
비유동비율(%) : 96
________________________
부채비율(%) : 96
________________________
유동부채비율(%) : 96
________________________
차입금의존도(%) : 96
________________________
이익잉여금비율(%) : 192
________________________
순운전자본비율(%) : 96
________________________
비유동장기적합률(%) : 96
________________________
이자보상배율(이자비용)(배) : 96
________________________
외화포지션(배) : 192
________________________
총자본순이익률(%) : 96
________________________
자기자본순이익률(%) : 96
________________________
경영자본순이익률(%) : 96
________________________
매출액순이익률(%) : 96
________________________
총자본정상영업이익률(%) : 96
________________________
자기자본정상영업이익률(%) : 96
________________________
매출액정상영업이익률(%) : 96
________________________
금융비용부담률(%) : 96
________________________
총자산회전율(배) : 192
________________________
매출채권회전률(배) : 96
________________________
당좌자산회전률(배) : 96
________________

(11248, 61)

## II-1. 재무비율 공통 NaN 제거
    - 96rows 제거 
    - 11,248 -> 11,152
    - 'Target' = 1 : 8 rows 제거

In [137]:
# 재무정보 columns : 공통적으로 결측치 존재 확인
main_df.loc[main_df['당좌자산회전률(배)'].isna()][['유동비율(%)', '당좌비율(%)', '유동부채비율(%)', '비유동비율(%)', '순운전자본비율(%)', '부채비율(%)',
       '차입금의존도(%)', '비유동장기적합률(%)', '자기자본순이익률(%)', '총자본순이익률(%)',
       '자기자본정상영업이익률(%)', '총자본정상영업이익률(%)', '경영자본순이익률(%)', '매출액순이익률(%)',
       '이자보상배율(이자비용)(배)', '금융비용부담률(%)', '매출액정상영업이익률(%)', '당좌자산회전률(배)',
       '재고자산회전률(배)', '자기자본회전률(배)', '경영자본회전률(배)', '매출채권회전률(배)', '매입채무회전률(배)',
       '유형자산회전율(배)', '매출액증가율(%)', '순이익증가율(%)', '정상영업이익증가율(%)', '유동자산증가율(%)',
       'log_평균총자산', '매출액(수익)(천원)', '이익잉여금비율(%)', '총자산회전율(배)',
       'CASH FLOW 대 부채비율(%)']]#.isna().sum()

,유동비율(%),당좌비율(%),유동부채비율(%),비유동비율(%),순운전자본비율(%),부채비율(%),차입금의존도(%),비유동장기적합률(%),자기자본순이익률(%),총자본순이익률(%),...,유형자산회전율(배),매출액증가율(%),순이익증가율(%),정상영업이익증가율(%),유동자산증가율(%),log_평균총자산,매출액(수익)(천원),이익잉여금비율(%),총자산회전율(배),CASH FLOW 대 부채비율(%)
64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16633,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17348,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17455,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [138]:
# 재무 공통 결측치 제거 
fin_index_drop = main_df.loc[main_df['당좌자산회전률(배)'].isna()].index
main_df.drop(index=fin_index_drop, inplace=True)

In [139]:
# 컬럼별 결측치 Check
for i in main_df.columns : 
    print(f'{i} : {main_df_copy_2[i].isna().sum()} -> {main_df[i].isna().sum()}')
    print('________________________')
    
main_df.shape

회사명 : 0 -> 0
________________________
거래소코드 : 0 -> 0
________________________
회계년도 : 0 -> 0
________________________
유동비율(%) : 96 -> 0
________________________
당좌비율(%) : 96 -> 0
________________________
비유동비율(%) : 96 -> 0
________________________
부채비율(%) : 96 -> 0
________________________
유동부채비율(%) : 96 -> 0
________________________
차입금의존도(%) : 96 -> 0
________________________
이익잉여금비율(%) : 192 -> 97
________________________
순운전자본비율(%) : 96 -> 0
________________________
비유동장기적합률(%) : 96 -> 0
________________________
이자보상배율(이자비용)(배) : 96 -> 0
________________________
외화포지션(배) : 192 -> 97
________________________
총자본순이익률(%) : 96 -> 0
________________________
자기자본순이익률(%) : 96 -> 0
________________________
경영자본순이익률(%) : 96 -> 0
________________________
매출액순이익률(%) : 96 -> 0
________________________
총자본정상영업이익률(%) : 96 -> 0
________________________
자기자본정상영업이익률(%) : 96 -> 0
________________________
매출액정상영업이익률(%) : 96 -> 0
________________________
금융비용부담률(%) : 96 -> 0
________________________
총자

(11152, 61)

In [140]:
main_df_copy_3 = main_df.copy()

## II-2. 시장정보(주가변수) 공통 NaN 제거
    - PBR, PER, PCR, 52주 베타, EV/EBITA
    - 11,152 -> 11,084 (68rows)
    - 제거 68rows 중 'Target' =1 : 12rows

In [141]:
# 68 rows 로 시장정보 NaN 값 갖는 row 동일 
main_df.loc[(main_df['PER'].isna())&(main_df['PER'].isna())&(main_df['PER'].isna()) & (main_df['EV/EBITDA(배)'].isna())& (main_df['52주베타(배)'].isna())]

# '1' : 12rows
# main_df.loc[(main_df['PER'].isna())&(main_df['PER'].isna())&(main_df['PER'].isna()) & (main_df['EV/EBITDA(배)'].isna())& (main_df['52주베타(배)'].isna())]['Target'].value_counts()

,회사명,거래소코드,회계년도,유동비율(%),당좌비율(%),비유동비율(%),부채비율(%),유동부채비율(%),차입금의존도(%),이익잉여금비율(%),...,최대주주변경횟수(회),감사법인변경횟수(회),업력(년),평균근속연수(년),직원수증감율(%),기간제비율(%),R&D집중도(%),수출기업여부(dummy),주거래은행,Target
6769,(주)영진코퍼레이션,053330,2014,39.68,39.68,0.00,0.00,0.00,42.83,-294.006991,...,2,1,13,NaN,-85.29,0.0,0.00,0,IBK기업은행,1
8631,(주)자비스,254120,2018,125.05,59.17,420.36,994.43,539.06,58.86,NaN,...,0,1,1,NaN,NaN,NaN,20.91,0,국민은행,0
9480,(주)카이노스메드,284620,2019,891.26,891.26,28.58,10.16,9.15,3.77,NaN,...,0,0,1,2.0,0.00,0.0,NaN,0,KEB하나은행,0
12003,(주)한국비엔씨,256840,2018,137.43,118.79,98.86,159.16,116.64,34.91,NaN,...,0,0,1,NaN,NaN,NaN,6.80,0,NH농협,0
12553,3노드디지탈그룹유한공사,900010,2012,112.82,94.91,104.01,232.47,202.50,35.54,21.267091,...,0,0,5,NaN,NaN,NaN,0.00,0,우리은행,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17342,헝셩그룹유한회사,900270,2018,411.65,402.21,25.18,24.89,24.22,12.81,69.810078,...,3,0,2,NaN,NaN,NaN,0.00,0,신한은행,0
17343,헝셩그룹유한회사,900270,2019,484.05,475.05,20.52,21.14,20.79,10.93,72.733622,...,4,0,3,NaN,NaN,NaN,0.00,0,신한은행,0
17344,헝셩그룹유한회사,900270,2020,515.18,506.99,23.44,18.53,18.46,10.56,74.670501,...,4,0,4,NaN,NaN,NaN,0.00,0,신한은행,0
17345,헝셩그룹유한회사,900270,2021,528.81,520.13,20.88,19.18,18.59,11.33,75.262694,...,4,0,5,NaN,NaN,NaN,0.00,0,신한은행,0


In [142]:
# 시장정보(주가변수) 공통 결측치 제거 
price_index_drop = main_df.loc[main_df['PER'].isna()].index
main_df.drop(index=price_index_drop, inplace=True)

In [143]:
# 컬럼별 결측치 Check
for i in main_df.columns : 
    print(f'{i} : {main_df_copy_3[i].isna().sum()} -> {main_df[i].isna().sum()}')
    print('________________________')

회사명 : 0 -> 0
________________________
거래소코드 : 0 -> 0
________________________
회계년도 : 0 -> 0
________________________
유동비율(%) : 0 -> 0
________________________
당좌비율(%) : 0 -> 0
________________________
비유동비율(%) : 0 -> 0
________________________
부채비율(%) : 0 -> 0
________________________
유동부채비율(%) : 0 -> 0
________________________
차입금의존도(%) : 0 -> 0
________________________
이익잉여금비율(%) : 97 -> 93
________________________
순운전자본비율(%) : 0 -> 0
________________________
비유동장기적합률(%) : 0 -> 0
________________________
이자보상배율(이자비용)(배) : 0 -> 0
________________________
외화포지션(배) : 97 -> 93
________________________
총자본순이익률(%) : 0 -> 0
________________________
자기자본순이익률(%) : 0 -> 0
________________________
경영자본순이익률(%) : 0 -> 0
________________________
매출액순이익률(%) : 0 -> 0
________________________
총자본정상영업이익률(%) : 0 -> 0
________________________
자기자본정상영업이익률(%) : 0 -> 0
________________________
매출액정상영업이익률(%) : 0 -> 0
________________________
금융비용부담률(%) : 0 -> 0
________________________
총자산회전율(배) : 97 -> 93


In [144]:
# 2018 이후 DF
# main_df.loc[main_df['회계년도'] >= 2018].to_excel('./main_df_2018~2022.xlsx')

In [145]:
# 표준산업분류 drop
main_df.drop('표준산업분류_중분류', axis=1, inplace=True)

## II-3. NaN값 : 0을 의미하는 컬럼 대체 
    - R&D집중도
    - 기간제비율


In [146]:
main_df.loc[main_df['R&D집중도(%)'].isna(), 'R&D집중도(%)'] = 0

In [147]:
main_df.loc[main_df['기간제비율(%)'].isna(), '기간제비율(%)'] = 0 

---

In [148]:
main_df_copy_4 = main_df.copy()

In [149]:
# K2_score기준 NaN 값 제거 : 95 rows 

main_df = main_df.loc[~(main_df['K2_score'].isna())]

In [150]:
# 컬럼별 결측치 Check
for i in main_df.columns : 
    print(f'{i} : {main_df_copy_4[i].isna().sum()} -> {main_df[i].isna().sum()}')
    print('________________________')

회사명 : 0 -> 0
________________________
거래소코드 : 0 -> 0
________________________
회계년도 : 0 -> 0
________________________
유동비율(%) : 0 -> 0
________________________
당좌비율(%) : 0 -> 0
________________________
비유동비율(%) : 0 -> 0
________________________
부채비율(%) : 0 -> 0
________________________
유동부채비율(%) : 0 -> 0
________________________
차입금의존도(%) : 0 -> 0
________________________
이익잉여금비율(%) : 93 -> 0
________________________
순운전자본비율(%) : 0 -> 0
________________________
비유동장기적합률(%) : 0 -> 0
________________________
이자보상배율(이자비용)(배) : 0 -> 0
________________________
외화포지션(배) : 93 -> 0
________________________
총자본순이익률(%) : 0 -> 0
________________________
자기자본순이익률(%) : 0 -> 0
________________________
경영자본순이익률(%) : 0 -> 0
________________________
매출액순이익률(%) : 0 -> 0
________________________
총자본정상영업이익률(%) : 0 -> 0
________________________
자기자본정상영업이익률(%) : 0 -> 0
________________________
매출액정상영업이익률(%) : 0 -> 0
________________________
금융비용부담률(%) : 0 -> 0
________________________
총자산회전율(배) : 93 -> 0
___

In [151]:
main_df_copy_5 = main_df.copy()

In [152]:
main_df_copy_5[(main_df_copy_5['K2_score'] == -(np.inf))]

,회사명,거래소코드,회계년도,유동비율(%),당좌비율(%),비유동비율(%),부채비율(%),유동부채비율(%),차입금의존도(%),이익잉여금비율(%),...,최대주주변경횟수(회),감사법인변경횟수(회),업력(년),평균근속연수(년),직원수증감율(%),기간제비율(%),R&D집중도(%),수출기업여부(dummy),주거래은행,Target
500,(주)네오이뮨텍,950220,2022,719.46,719.46,18.29,16.86,13.70,3.47,-111.171316,...,3,0,1,1.720000,27.78,0.00,0.0,0,KEB하나은행,0
2218,(주)메드팩토,235980,2020,1989.92,1989.92,16.99,4.94,4.42,1.02,-116.574736,...,1,1,0,1.600000,47.22,1.89,0.0,0,IBK기업은행,0
2219,(주)메드팩토,235980,2021,12895.44,12895.44,16.67,206.24,2.25,0.14,-200.545389,...,1,1,1,1.900000,30.19,1.45,0.0,0,IBK기업은행,0
2220,(주)메드팩토,235980,2022,137.82,137.82,22.76,233.16,225.23,64.59,-332.043679,...,1,1,2,2.100000,-10.14,0.00,0.0,0,IBK기업은행,0
2664,(주)박셀바이오,323990,2021,7187.70,7187.70,10.24,3.98,1.30,0.70,-73.813365,...,2,1,1,2.500000,116.00,7.41,0.0,0,광주은행,0
2665,(주)박셀바이오,323990,2022,6806.38,6806.38,13.99,3.58,1.32,0.44,-112.133719,...,3,1,2,3.000000,-12.96,6.38,0.0,0,광주은행,0
3787,(주)셀리드,299660,2020,1242.97,1242.97,77.02,4.45,2.21,0.19,-9.104799,...,3,2,1,1.666667,46.88,0.00,0.0,0,신한은행,0
10353,(주)큐리언트,115180,2020,2179.12,2179.12,3.61,6.76,4.73,1.82,-280.248742,...,3,2,4,4.270000,0.00,0.00,inf,0,신한은행,1
11066,(주)파멥신,208340,2019,5190.04,5190.04,37.16,138.03,3.87,33.84,-63.109035,...,2,3,1,3.333333,69.57,0.00,0.0,0,KEB하나은행,0
13752,바이젠셀(주),308080,2022,836.72,836.72,32.89,10.85,9.32,7.46,-62.826395,...,1,1,1,1.690000,53.85,0.00,0.0,0,신한은행,0


In [153]:
main_df['Target'].value_counts()

0    10055
1      934
Name: Target, dtype: int64

In [154]:
# 제거 
main_df_2 = main_df.loc[~(main_df['평균근속연수(년)'].isna())].isna().sum()

In [155]:
# R&D 집중도 : inf & 음수값  
# main_df[main_df['R&D집중도(%)'] < 0] 
main_df['R&D집중도(%)'].describe(percentiles=[ .005 ,.995])
main_df.loc[main_df.loc[main_df['R&D집중도(%)'] > 483.7808, 'R&D집중도(%)'].index][['회계년도']]
# main_df.loc[main_df['R&D집중도(%)'] < 0, 'R&D집중도(%)'] = 0


,회계년도
44,2016
1831,2014
3788,2021
3789,2022
4266,2021
4267,2022
4345,2021
5276,2021
5277,2022
6393,2018


In [156]:
# 이상치 - 직원수증감율

# main_df[main_df['직원수증감율(%)']>10000] # 13rows
main_df['직원수증감율(%)'].isna().sum()


18

---

In [157]:
# eda_df = main_df.drop(['R&D집중도(%)', 'K2_score'], axis=1)

In [158]:
# EDA용 dtype변경
# eda_df['Target'] = eda_df['Target'].astype('int')

In [159]:
# EDA html

# import sweetviz

# eda_df_train = eda_df.loc[(eda_df['회계년도'] >= 2011) & (eda_df['회계년도'] < 2020)]
# eda_df_test = eda_df.loc[(eda_df['회계년도'] >= 2020) & (eda_df['회계년도'] < 2023)]

# df_report = sweetviz.compare([eda_df_train, 'Train'], [eda_df_test, 'Test'], 'Target')

# df_report.show_html('./EDA_final.html')



# III. ``